# 5.2. Tratando as *Features* extraídas

* **Input**: Dados com as respostas de LLM não tratadas

* **Output**: Respostas de LLM tratas + variável com True/False da alternativa

## 5.2.1. Extração da tabela original

In [582]:
# Importando Dependências
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [584]:
# Leitura dos dados
enem_data = pd.read_pickle("../data/final/enem_data_embeddings.pkl")
enem_data.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,distratores_embbedings_word2vec_100,similaridade_enunciado_gabarito_100,similaridade_enunciado_distratores_100,similaridade_gabarito_distratores_100,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[-0.041815996, -0.06492707, 0.0900044, -0.0498...",0.141353,0.516898,0.151251,"[-0.017395772, -0.004402813, -0.046908338, 0.0...","[-0.16301225, -0.142375, 0.012171499, -0.00457...","[-0.041815996, -0.06492707, 0.0900044, -0.0498...",0.141353,0.516898,0.151251
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[0.05533243, -0.039066706, -0.059438962, 0.019...",0.422576,0.313321,0.423611,"[-0.051949076, 0.03128451, -0.072241075, 0.028...","[-0.18672037, 0.04755275, -0.17406088, -0.0398...","[0.05533243, -0.039066706, -0.059438962, 0.019...",0.422576,0.313321,0.423611
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[0.06667993, 0.015803386, 0.029443847, 0.11312...",0.427063,0.471643,0.602545,"[0.026049094, 0.043251127, -0.029757027, -0.00...","[0.011740998, 0.07592634, -0.016144669, 0.1384...","[0.06667993, 0.015803386, 0.029443847, 0.11312...",0.427063,0.471643,0.602545
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[-0.07921684, 0.10788742, 0.10466367, -0.04121...",0.320939,0.482462,0.421501,"[-0.005251272, 0.039871164, -0.018038195, 0.06...","[-0.073669255, 0.12609875, 0.063628, 0.1261957...","[-0.07921684, 0.10788742, 0.10466367, -0.04121...",0.320939,0.482462,0.421501
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[-0.02205533, 0.08129784, 0.10641242, -0.05484...",0.304050,0.501346,0.411802,"[0.01738678, 0.0033733728, -0.027247325, 0.002...","[0.11709199, -0.16062833, 0.082869664, -0.1195...","[-0.02205533, 0.08129784, 0.10641242, -0.05484...",0.304050,0.501346,0.411802


## 5.2.2. Extração das respostas do Llama

In [585]:
# leitura do csv

df_llama = pd.read_csv('../data/final/llama_extraction_csv.csv') 
df_llama.head()

,Unnamed: 0,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec,resposta_llamma,acerto_llamma
0,0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,[ 6.11259453e-02 -4.80512679e-02 -5.90382852e-...,[ 0.0875405 0.14673425 0.17936975 -0.033228...,C,True
1,1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,[ 9.6771289e-03 1.1954352e-01 2.1252066e-02 ...,[ 1.61291629e-01 2.37609118e-01 8.02588761e-...,D,True
2,2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,[ 0.06814931 -0.02395453 0.02739678 -0.003347...,[ 0.26195732 -0.13758467 -0.16108766 0.071330...,D.,False
3,3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,[ 0.01932281 -0.03491368 -0.00591799 0.012698...,[ 0.253879 -0.00519249 -0.02825475 0.109827...,E.,False
4,4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,[-3.24988738e-02 8.80865678e-02 -4.07014266e-...,[ 0.248065 -0.15562867 -0.114118 -0.121275...,C,True


In [586]:
# dropando coluna não utilizada

df_llama.drop('acerto_llamma', axis=1, inplace = True) 

In [587]:
# utilizando regex para padronizar respostas que a LLM deu

df_llama['resposta_llamma_limpo'] = df_llama['resposta_llamma'].str.findall(r'\b([A-E])\b').str[-1]

In [588]:
# gerando variável true/false de acertos do modelo Llama

df_llama["acerto_llamma"] = df_llama[["gabarito", "resposta_llamma_limpo"]].apply(
    lambda row: row["gabarito"] == row["resposta_llamma_limpo"], axis=1
)

In [589]:
# quantidade de acertos do Llama

df_llama["acerto_llamma"].sum()

154

In [590]:
# pegando apenas as colunas de interesse e as necessárias para o merge

df_llama = df_llama[['numero_questao','ANO','resposta_llamma_limpo','acerto_llamma']]

In [591]:
# padronizando nome

df_llama.rename(columns={'resposta_llamma_limpo': 'resposta_llama'}, inplace=True)


In [592]:
# verificando tabela final

display(df_llama)

,numero_questao,ANO,resposta_llama,acerto_llamma
0,1,2017,C,True
1,2,2017,D,True
2,3,2017,D,True
3,4,2017,E,True
4,5,2017,C,True
5,6,2017,A,True
6,7,2017,D,True
7,8,2017,B,True
8,9,2017,E,True
9,10,2017,E,False


## 5.2.3. Extração das respostas do Deepseek

In [593]:
# leitura do csv

df_deepseek = pd.read_csv('../data/final/deepseek_extraction_csv.csv') 
df_deepseek.head()

,Unnamed: 0,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec,resposta_deepseek,acerto_deepseek
0,0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,[ 6.11259453e-02 -4.80512679e-02 -5.90382852e-...,[ 0.0875405 0.14673425 0.17936975 -0.033228...,A resposta correta é: C,False
1,1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,[ 9.6771289e-03 1.1954352e-01 2.1252066e-02 ...,[ 1.61291629e-01 2.37609118e-01 8.02588761e-...,D,True
2,2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,[ 0.06814931 -0.02395453 0.02739678 -0.003347...,[ 0.26195732 -0.13758467 -0.16108766 0.071330...,D,True
3,3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,[ 0.01932281 -0.03491368 -0.00591799 0.012698...,[ 0.253879 -0.00519249 -0.02825475 0.109827...,E,True
4,4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,[-3.24988738e-02 8.80865678e-02 -4.07014266e-...,[ 0.248065 -0.15562867 -0.114118 -0.121275...,C,True


In [594]:
# dropando coluna não utilizada

df_deepseek.drop('acerto_deepseek', axis=1, inplace = True) 

In [1]:
# aqui visualizamos e tratamos manualmente algumas respostas mais complexas que o Deepseek forneceu

df_deepseek.loc[10, 'resposta_deepseek']

NameError: name 'df_deepseek' is not defined

In [596]:
df_deepseek.loc[10, 'resposta_deepseek'] = 'B'

In [597]:
df_deepseek.loc[41, 'resposta_deepseek']

'Vamos analisar as informações fornecidas e as alternativas:\n\n1. O enunciado descreve as temperaturas mínimas em diferentes regiões do Brasil e os índices de umidade relativa do ar.\n2. A pergunta específica é sobre o fator climático determinante para explicar os altos índices de umidade nas regiões Nordeste e Sul.\n\nAnalisando as alternativas:\n- A) Altitude: Não é o fator principal, pois ambas as regiões têm áreas de baixa altitude.\n- B) Vegetação: Embora possa influenciar, não é o principal fator para a umidade nessas regiões.\n- C) Massas de ar: Esta é a resposta correta, pois as massas de ar úmidas (como a massa Tropical Atlântica no Nordeste e a Polar Atlântica no Sul) são responsáveis pela alta umidade e precipitação nessas regiões.\n- D) Correntes marítimas: Influenciam, mas não são o fator principal.\n- E) Continentalidade: Não se aplica, pois ambas as regiões têm forte influência oceânica.\n\nA resposta correta é: C'

In [598]:
df_deepseek.loc[41, 'resposta_deepseek'] = 'C'

In [599]:
df_deepseek.loc[98, 'resposta_deepseek']

'Vamos analisar cada alternativa:\n\nA) Ciclo hidrológico - Não está relacionado à formação de continentes.\nB) Processo erosivo - Também não explica a formação de um supercontinente.\nC) Estrutura geológica - Esta é a correta, pois a formação e movimentação de continentes está diretamente relacionada aos processos geológicos e tectônicos.\nD) Índice pluviométrico - Não tem relação com a formação de continentes.\nE) Pressão atmosférica - Também não está relacionada.\n\nA resposta correta é: C'

In [600]:
df_deepseek.loc[98, 'resposta_deepseek'] = 'C'

In [601]:
df_deepseek.loc[103, 'resposta_deepseek']

'Vamos analisar o enunciado e as alternativas:\n\nO texto descreve como Newton, ao observar a queda da maçã, passou de uma explicação superficial (animais derrubando) para uma busca por princípios fundamentais (força subjetiva causando a queda). Isso representa o núcleo do método científico moderno - ir além das aparências para descobrir leis universais.\n\nAnalisando as alternativas:\nA) Falsificação - não é o foco (Newton não está testando/excluindo hipóteses)\nB) Negação da observação - incorreto (ele parte da observação)\nC) Proposição de hipóteses - parcialmente correto, mas não capta a essência\nD) Contemplação - muito passivo (Newton está ativamente buscando princípios)\nE) Universalização - CORRETO (ele busca uma lei geral a partir do caso específico)\n\nA resposta correta é: E'

In [602]:
df_deepseek.loc[103, 'resposta_deepseek'] = 'E'

In [603]:
df_deepseek.loc[127, 'resposta_deepseek']

'Vamos analisar cada alternativa:\n\nA) "promoção de atos ecumênicos" - O texto não menciona reconciliação entre diferentes religiões.\nB) "fomento de orientações bíblicas" - Embora haja adaptação, não é o foco principal.\nC) "apropriação de cerimônias seculares" - Corresponde à absorção de rituais pagãos.\nD) "retomada de ensinamentos apostólicos" - Não há referência a ensinos originais dos apóstolos.\nE) "ressignificação de rituais fundamentalistas" - O termo "fundamentalistas" não se aplica ao contexto.\n\nA alternativa que melhor descreve o processo de assimilação de rituais pagãos (como as fogueiras) pela Igreja é:\n\nC'

In [604]:
df_deepseek.loc[127, 'resposta_deepseek'] = 'C'

In [605]:
df_deepseek.loc[139, 'resposta_deepseek']

'Vamos analisar o texto e as alternativas:\n\nO texto descreve um modelo de planejamento urbano que:\n- Incentiva o crescimento urbano a qualquer custo\n- Vê as cidades como máquinas de produzir riqueza\n- Foca em "apertar a máquina" (otimização econômica)\n\nIsso caracteriza um modelo:\n- Não participativo (exclui gestão popular - A)\n- Não sustentável (exclui B)\n- Não focado em bem-estar social (exclui C)\n- Centralizado no poder econômico, não governamental (exclui D)\n\nA alternativa que melhor se encaixa é:\nE - ampliação da participação empresarial (o texto enfatiza a lógica empresarial de produção de riqueza)\n\nResposta: E'

In [606]:
df_deepseek.loc[139, 'resposta_deepseek'] = 'E'

In [607]:
df_deepseek.loc[158, 'resposta_deepseek']

'Vamos analisar o texto e as alternativas:\n\nO texto descreve a poluição do rio Tietê, especificamente mencionando:\n1. A água não tem oxigênio suficiente para sustentar vida\n2. A poluição ocorre ao longo de uma área urbana/metropolitana\n3. A causa está relacionada com a falta de oxigênio na água\n\nAnalisando as alternativas:\nA) Não menciona sítio urbano - irrelevante\nB) Extinção da vegetação não é a causa direta da falta de oxigênio\nC) Carga de dejetos (esgoto) é a principal causa da falta de oxigênio em rios urbanos\nD) Captação de água pluvial não causa falta de oxigênio\nE) Defensivos agrícolas não são mencionados como causa\n\nA causa mais direta para a falta de oxigênio em rios urbanos é o lançamento de esgoto (carga orgânica que consome oxigênio na decomposição).\n\nResposta correta: C'

In [608]:
df_deepseek.loc[158, 'resposta_deepseek'] = 'C'

In [609]:


df_deepseek.loc[176, 'resposta_deepseek']

'Vamos analisar cada alternativa:\n\nA) A estagnação de direitos adquiridos e o anacronismo da legislação vigente são mencionados, mas não explicam a desigualdade de oportunidades.\n\nB) A manutenção do status quo gerencial e dos padrões de socialização familiar são fatores que perpetuam a desigualdade de oportunidades, pois mantêm as estruturas que limitam as mulheres.\n\nC) A desestruturação da herança patriarcal e mudanças no perfil ocupacional seriam positivas, não explicando a desigualdade.\n\nD) A disputa na composição sindical e presença política são relevantes, mas não são a causa central da desigualdade de oportunidades.\n\nE) As exigências de aperfeiçoamento profissional e habilidades gerenciais são consequências, não causas da desigualdade.\n\nA alternativa que melhor explica a persistência da desigualdade de oportunidades é a B, pois aponta para a manutenção das estruturas sociais e gerenciais que perpetuam a desigualdade.\n\nB'

In [610]:
df_deepseek.loc[176, 'resposta_deepseek'] = 'B'

In [611]:
df_deepseek.loc[186, 'resposta_deepseek']

'Pela análise do texto, a situação descrita refere-se a pessoas que não têm nacionalidade reconhecida por nenhum país, o que caracteriza a condição de apátrida. Portanto, a alternativa correta é:\n\nB'

In [612]:
df_deepseek.loc[186, 'resposta_deepseek'] = 'B'

In [613]:
df_deepseek.loc[203, 'resposta_deepseek']

'Analisando o enunciado, o fator determinante para a instalação das refinarias de petróleo é a proximidade a espaços com infraestrutura de circulação, pois isso facilita o transporte tanto da matéria-prima quanto dos produtos finais.\n\nA resposta correta é: E'

In [614]:
df_deepseek.loc[203, 'resposta_deepseek'] = 'E'

In [615]:
df_deepseek.loc[239, 'resposta_deepseek']

'O texto discute a natureza do perdão, destacando que ele não é garantido e depende da autonomia de quem o concede. A alternativa que melhor reflete essa ideia é a que enfatiza a autonomia dos indivíduos nas relações interpessoais. Portanto, a resposta correta é:\n\nE'

In [616]:
df_deepseek.loc[239, 'resposta_deepseek'] = 'E'

In [617]:
df_deepseek.loc[244, 'resposta_deepseek']

'Analisando o texto e o contexto histórico da Declaração Balfour, o objetivo declarado era estabelecer um lar nacional para o povo judeu na Palestina, sem prejudicar os direitos das comunidades não judias. No entanto, o processo resultou em conflitos territoriais entre judeus e árabes, demonstrando a incapacidade do governo britânico de mediar esses conflitos de forma efetiva.\n\nA alternativa correta é: C'

In [618]:
df_deepseek.loc[244, 'resposta_deepseek'] = 'C'

In [619]:
# agora, tratamos de forma mais geral as respostas menos complexas

df_deepseek['resposta_deepseek_limpo'] = df_deepseek['resposta_deepseek'].str.findall(r'\b([A-E])\b').str[-1]

In [620]:
# gerando variável de true/false de acertos do modelo Deepseek

df_deepseek["acerto_deepseek"] = df_deepseek[["gabarito", "resposta_deepseek_limpo"]].apply(
    lambda row: row["gabarito"] == row["resposta_deepseek_limpo"], axis=1
)

In [621]:
# quantidade de acertos do Deepseek

df_deepseek["acerto_deepseek"].sum()

223

In [622]:
# pegando apenas as colunas de interesse e as necessárias para o merge

df_deepseek = df_deepseek[['numero_questao','ANO','resposta_deepseek_limpo','acerto_deepseek']]

In [623]:
# padronizando nome

df_deepseek.rename(columns={'resposta_deepseek_limpo': 'resposta_deepseek'}, inplace=True)


In [624]:
# verificando tabela final

display(df_deepseek)

,numero_questao,ANO,resposta_deepseek,acerto_deepseek
0,1,2017,C,True
1,2,2017,D,True
2,3,2017,D,True
3,4,2017,E,True
4,5,2017,C,True
5,6,2017,A,True
6,7,2017,D,True
7,8,2017,B,True
8,9,2017,E,True
9,10,2017,A,True


## 5.2.4. Extração das respostas do Deepseek

In [625]:
# inserindo as novas variáveis do Llama no dataset original

df_merged = pd.merge(
    enem_data,                 
    df_llama,                 
    on=['numero_questao', 'ANO'],  
    how='left'           
)

In [626]:
# inserindo as novas variáveis do Deepseek no dataset original

df_merged = pd.merge(
    df_merged,                 
    df_deepseek,                 
    on=['numero_questao', 'ANO'],  
    how='left'           
)

In [627]:
# resultado final

display(df_merged)

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50,resposta_llama,acerto_llamma,resposta_deepseek,acerto_deepseek
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[-0.017395772, -0.004402813, -0.046908338, 0.0...","[-0.16301225, -0.142375, 0.012171499, -0.00457...","[-0.041815996, -0.06492707, 0.0900044, -0.0498...",0.141353,0.516898,0.151251,C,True,C,True
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[-0.051949076, 0.03128451, -0.072241075, 0.028...","[-0.18672037, 0.04755275, -0.17406088, -0.0398...","[0.05533243, -0.039066706, -0.059438962, 0.019...",0.422576,0.313321,0.423611,D,True,D,True
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[0.026049094, 0.043251127, -0.029757027, -0.00...","[0.011740998, 0.07592634, -0.016144669, 0.1384...","[0.06667993, 0.015803386, 0.029443847, 0.11312...",0.427063,0.471643,0.602545,D,True,D,True
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[-0.005251272, 0.039871164, -0.018038195, 0.06...","[-0.073669255, 0.12609875, 0.063628, 0.1261957...","[-0.07921684, 0.10788742, 0.10466367, -0.04121...",0.320939,0.482462,0.421501,E,True,E,True
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[0.01738678, 0.0033733728, -0.027247325, 0.002...","[0.11709199, -0.16062833, 0.082869664, -0.1195...","[-0.02205533, 0.08129784, 0.10641242, -0.05484...",0.304050,0.501346,0.411802,C,True,C,True
5,6,A,2.04543,0.74139,0.20814,2017,Descrição da fotografia: É retratada uma mulhe...,A: ambiguidade do trabalho doméstico exercido ...,ambiguidade do trabalho doméstico exercido pel...,integração dos escravos aos valores das classe...,...,"[-0.060001582, 0.03424079, -0.010523678, 0.077...","[0.011028358, -0.024059093, 0.010935817, -0.03...","[-0.07814517, -0.0028704864, 0.022850823, -0.0...",0.425350,0.443914,0.572307,A,True,A,True
6,7,D,4.02261,0.96775,0.17554,2017,A conﬁguração do espaço urbano da região do En...,A: expansão vertical; B: polarização naciona...,segregação socioespacial,expansão vertical; polarização nacional.; eman...,...,"[0.00042285025, -0.052510247, 0.0038767974, -0...","[-0.085658, 0.108736, 0.044327497, -0.08828749...","[0.107376374, -0.09405625, 0.08613763, 0.01331...",0.271435,0.422045,0.412837,D,True,D,True
7,8,B,1.33884,2.80117,0.16665,2017,"México, Colômbia, Peru e Chile decidiram segui...",A: promover a livre circulação de trabalhadore...,fomentar a competitividade no mercado externo,promover a livre circulação de trabalhadores; ...,...,"[0.055130266, -0.03423147, 0.011841437, 0.0882...","[0.07473776, -0.016530499, 0.255476, -0.086876...","[-0.03886994, -0.03993039, 0.13154513, -0.1846...",0.288048,0.297197,0.559286,B,True,B,True
8,9,E,3.15884,0.92459,0.09500,2017,Descrição da tabela: Indica a tipologia de áre...,A: erosão laminar; B: intemperismo físico; C...,recarga dos aquíferos,erosão laminar; intemperismo físico; ench

In [ ]:
# salvando csv

df_merged.to_csv('enem_data_embeddings_v2.csv')